In [ ]:
# External Dataset Integration - Tiny ImageNet
# Import our organized helper functions
import sys
sys.path.append('../src')

from src.data import DataPreprocessor
from src.utils import setup_tiny_imagenet_experiment
import matplotlib.pyplot as plt

In [ ]:
# Setup Tiny ImageNet integration using our helper function
class_mappings = {
    'n03706229': 8,  # magnetic compass -> class 8  
    'n04456115': 9   # torch -> class 9
}

# This will download, extract, and prepare the external dataset
imagenet_data = setup_tiny_imagenet_experiment(
    output_dir='./',
    class_mappings=class_mappings,
    image_shape=(160, 160, 3)
)

print("Tiny ImageNet integration completed!")
print("Available data components:", list(imagenet_data.keys()))
print(f"Training samples: {len(imagenet_data['x_train_m'])}")
print(f"Test samples: {len(imagenet_data['x_test_m'])}")

In [ ]:
# Visualize some samples from the external dataset
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Tiny ImageNet Samples for M-Dataset')

for i in range(10):
    row = i // 5
    col = i % 5
    
    # Get sample
    sample_image = imagenet_data['x_train_m'][i]
    sample_label = imagenet_data['y_train_m'][i]
    
    # Display
    axes[row, col].imshow(sample_image.astype('uint8'))
    axes[row, col].set_title(f'Class {np.argmax(sample_label)}')
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print("External dataset samples visualized!")
print("These images will serve as the M-dataset in on-device training experiments")

In [ ]:
#!pip install tensorflow pillow
!pip install wget


In [ ]:
from tensorflow.keras.models import load_model

# Load the model saved in Keras HDF5 format
model = load_model("/kaggle/input/cifar9_95acc/tensorflow2/cifar9_95acc/1/mnv2_cifar9_160_fbn_4.keras")

import os, sys, wget
from zipfile import ZipFile

url = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
tiny_imgdataset = wget.download(url, out=os.getcwd())

# Extract the downloaded zip file
zip_file = os.path.join(os.getcwd(), "tiny-imagenet-200.zip")

if os.path.exists(zip_file):
    with ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(os.getcwd())
    print("Extraction completed.")
else:
    print("Zip file not found.")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import glob

# Set parameters
IMAGE_SHAPE = (160, 160, 3)
IMG_SIZE = 160
BATCH_SIZE = 128
BUFFER_SIZE = BATCH_SIZE * 10
AUTO = tf.data.AUTOTUNE

# Define preprocessing layers
preprocessing = keras.Sequential(
    [
        layers.Rescaling(1./255.0, offset=0),
        layers.Resizing(IMAGE_SHAPE[0], IMAGE_SHAPE[1], interpolation='bilinear')
    ],
    name="preprocessing",
)

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.15),
        layers.RandomContrast(factor=0.1),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)
def load_images_from_folder(folder, label):
    image_files = glob.glob(os.path.join(folder, "*.JPEG"))
    print(f"Found {len(image_files)} images")
    images = []
    labels = []
    for img_file in image_files:
        img = tf.io.read_file(img_file)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [IMAGE_SHAPE[0], IMAGE_SHAPE[1]])  # Resize image
        images.append(img)
        labels.append(label)
    return images, labels


    
train_folder = "/kaggle/working/tiny-imagenet-200/train/n03706229/images" #magnetic compass- πυξίδα
train_images, train_labels = load_images_from_folder(train_folder, 9)




In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

# Assuming train_images is a list of TensorFlow tensors
# Get the first image tensor from the list
img_tensor = train_images[3]

# Convert the tensor to a numpy array and then to a PIL image
img_array = img_to_array(img_tensor)
img = array_to_img(img_array)

# Plot the image
plt.figure(figsize=(5, 5))
plt.imshow(img)
plt.axis('off')  # Hide axis labels
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
# Load the model saved in Keras HDF5 format
model = load_model("/kaggle/input/cifar9_95acc/tensorflow2/cifar9_95acc/1/mnv2_cifar9_160_fbn_4.keras")

NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# create n- and m-datasets
def create_datasets(x, y, n_classes):

    x_n = []
    y_n = []
    
    for x_, y_ in zip (x, y):
        if y_ in n_classes:
            x_n.append(x_)
            y_n.append(y_)
    return np.array(x_n), np.array(y_n)
    
n_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8]
x_train_n, y_train_n= create_datasets(x_train, y_train, n_classes)
x_test_n, y_test_n = create_datasets(x_test, y_test, n_classes)

# Preprocess ImageNet samples
x_m = []
y_m = []
for m in  train_images:
    x_m.append(m)
    y_m.append(train_labels[0])
x_m = np.array(x_m)
y_m = np.array(y_m)
train_ratio = 0.8
test_ratio = 0.2

# Shuffle and split the data
x_train_m, x_test_m, y_train_m, y_test_m = train_test_split(
    x_m, y_m, 
    test_size=test_ratio, 
    random_state=42  # For reproducibility
)


y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

y_train_n = tf.keras.utils.to_categorical(y_train_n, num_classes=10)
y_test_n = tf.keras.utils.to_categorical(y_test_n, num_classes=10)

y_train_m = tf.keras.utils.to_categorical(y_train_m, num_classes=10)
y_test_m = tf.keras.utils.to_categorical(y_test_m, num_classes=10)

# preprocessing
IMAGE_SHAPE = (160, 160, 3)

preprocessing = keras.Sequential(
    [
        layers.Rescaling(1./255.0, offset=0),
        layers.Resizing(IMAGE_SHAPE[0], IMAGE_SHAPE[1], interpolation='bilinear')
    ],
    name="preprocessing",
)

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.15),
        layers.RandomContrast(factor=0.1),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)

IMG_SIZE = 160
BATCH_SIZE = 128
BUFFER_SIZE = BATCH_SIZE * 10
AUTO = tf.data.AUTOTUNE

# original
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_train = ds_train.shuffle(ds_train.cardinality()).batch(BATCH_SIZE).map(lambda x, y: (data_augmentation(preprocessing(x)), y)).prefetch(AUTO)

ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
ds_test = ds_test.batch(BATCH_SIZE).map(lambda x, y: (preprocessing(x), y)).prefetch(AUTO)

# n-dataset
ds_train_n = tf.data.Dataset.from_tensor_slices((x_train_n, y_train_n))
ds_train_n = ds_train_n.shuffle(ds_train_n.cardinality()).batch(BATCH_SIZE).map(lambda x, y: (data_augmentation(preprocessing(x)), y)).prefetch(AUTO)

ds_test_n = tf.data.Dataset.from_tensor_slices((x_test_n, y_test_n))
ds_test_n = ds_test_n.batch(BATCH_SIZE).map(lambda x, y: (preprocessing(x), y)).prefetch(AUTO)

# m-dataset
ds_train_m = tf.data.Dataset.from_tensor_slices((x_train_m, y_train_m))
ds_train_m = ds_train_m.shuffle(ds_train_m.cardinality()).batch(BATCH_SIZE).map(lambda x, y: (data_augmentation(preprocessing(x)), y)).prefetch(AUTO)

ds_test_m = tf.data.Dataset.from_tensor_slices((x_test_m, y_test_m))
ds_test_m = ds_test_m.batch(BATCH_SIZE).map(lambda x, y: (preprocessing(x), y)).prefetch(AUTO)

In [ ]:
ds_test_n = ds_test_n.unbatch().batch(1)
ds_train_n = ds_train_n.unbatch().batch(1)
ds_train_m = ds_train_m.unbatch().batch(1)


In [ ]:
num_batches = sum(1 for _ in ds_train_m)
print("Number of batches in ds_test_m:", num_batches)

# Calculate the total number of examples
# Iterate through the dataset and count the total number of examples
total_examples = sum(len(batch[0]) for batch in ds_train_m)
print("Total number of examples in ds_test_m:", total_examples)

In [ ]:
def calculate_bvsb(probs):
    sorted_probs = np.sort(probs, axis=1)[:, ::-1]
    bvsb = sorted_probs[:, 0] - sorted_probs[:, 1]
    return bvsb

def save_images(images, output_dir, image_format='PNG'):
    os.makedirs(output_dir, exist_ok=True)
    
    for idx, image in enumerate(images):
        image = np.clip(image, 0.0, 1.0)
        image_uint8 = (image.squeeze() * 255.0).astype(np.uint8)
        if image_uint8.shape[0] == 3:
            image_uint8 = np.transpose(image_uint8, (1, 2, 0))
        image_pil = Image.fromarray(image_uint8)
        temp_image_path = os.path.join(output_dir, f'image_{idx}.{image_format.lower()}')
        image_pil.save(temp_image_path, format=image_format)


In [ ]:
import numpy as np
import os
import absl.logging
from PIL import Image
import os
import zipfile
m_samples = []
i =0 
for x, y in ds_train_m:
    m_samples.append(x.numpy())
    if i == 50:
        break
    i+=1 

output_zip_path = '/kaggle/working/m_samples_imagenet'
save_images(m_samples, output_zip_path, image_format='PNG')

In [11]:
import shutil
import os

# Specify the directory you want to zip (for example, 'my_directory')
dir_to_zip = "/kaggle/working/m_samples_imagenet"

# Output zip file path
output_zip = "/kaggle/working/m_samples_imagenet.zip"

# Create a zip file of the directory
shutil.make_archive(output_zip.replace(".zip", ""), 'zip', dir_to_zip)

from IPython.display import FileLink

# Provide a download link for the zip file
FileLink(output_zip)


/kaggle/working/m_samples_imagenet.zip

In [ ]:
from sklearn.model_selection import train_test_split
n_labels = [0,1,2,3, 4, 5, 6, 7,8]
x_test = []
y_test = []
labels = []
for x,y in ds_test_n:  
    x_test.append(x.numpy())
    y_test.append(np.argmax(y))
  
x_test = np.array(x_test,dtype=np.float32)
y_test = np.array(y_test,dtype=np.float32)
test_size = 270
random_seed = 42  
_,x_sampled,_, y_sampled = train_test_split(
    x_test, y_test,test_size = test_size , stratify=y_test, random_state=random_seed
)

y=  tf.keras.utils.to_categorical(y_sampled, num_classes=10)

ds_test_sampled = tf.data.Dataset.from_tensor_slices((x_sampled, y))
def reshape_y(x, y):
    y = tf.reshape(y, (1, 10))
    return x, y
ds_test_stratified = ds_test_sampled.map(reshape_y)
for i in n_labels:
    arr = []
    count = 0 
    n_labels_copy = n_labels.copy()
    n_labels_copy.remove(i)
    n_labels_ = tf.keras.utils.to_categorical(n_labels_copy, num_classes=10)
    for x,y in ds_test_stratified:
        if  not any(np.array_equal(y[0].numpy(), n) for n in n_labels_):
            count+=1
            arr.append(x.numpy())
    #output_zip_path =f'/kaggle/working/ds_test_strartified_{i}'
   # save_images(arr, output_zip_path, image_format='PNG')
    print("samples of class {} = {}".format(i,count))
            
    
_, acc_sampled = model.evaluate( ds_test_stratified, verbose=0)
acc_sampled



In [ ]:
from sklearn.model_selection import train_test_split
import random
from collections import Counter
from sklearn.utils import resample


n_labels = [0,1,2,3, 4, 5, 6, 7,8]
accuracies_sampled = []
sampled_datasets = []
def reshape_y(x, y):
    y = tf.reshape(y, (1, 10))
    return x, y

for i in n_labels:
    n_labels_copy = n_labels.copy()
    n_labels_copy.remove(i)
    n_labels_ = tf.keras.utils.to_categorical(n_labels_copy, num_classes=10)
    samples_true = []
    samples_false = []
    x_t = []
    y_t = []
    #find the images that model classifies correct and false 
    for x,y in ds_test_n:
        if  not any(np.array_equal(y[0].numpy(), n) for n in n_labels_):
            x_t.append(x.numpy())
            y_t.append(y)
            prediction = model.predict(x,verbose=0).astype(np.float64)
            if(np.argmax(y)==np.argmax(prediction)):
                samples_true.append((x,np.max(prediction))) # save top-1 softmax
            else:
                samples_false.append((x,prediction))
    
    # plot histogram of top-1 softmax 

    top_1 = [top_1 for _,top_1 in samples_true]
    """ 
    plt.hist(top_1,bins=20,edgecolor='black')
    plt.title(f'Top-1 softmax Histogram for class {i}')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.savefig(f'top_1_hist_class_{i}.png')
    plt.show()
    """
    num_samples = 29  # Number of samples to keep
    bins = np.histogram_bin_edges(top_1, bins=20)  # Define bin edges
    bin_indices = np.digitize(top_1, bins)  # Assign scores to bins

    sub_samples = []

    for b in range(1, len(bins)):
        bin_samples = [(x, score) for (x, score), bin_idx in zip(samples_true, bin_indices) if bin_idx == b]
        if bin_samples:
            # Subsample from the bin to keep the same distribution
            subsampled_bin_samples = resample(bin_samples, replace=False, n_samples=int(np.round(len(bin_samples) * num_samples / len(top_1))))
            sub_samples.extend(subsampled_bin_samples)
            
    if len( sub_samples) > num_samples:
        sub_samples = resample(sub_samples, replace=False, n_samples=num_samples)
    elif len(sub_samples) < num_samples:
        additional_samples = resample(samples_true, replace=False, n_samples=num_samples - len(sub_samples))
        sub_samples.extend(additional_samples)

    # Now `stratified_samples` contains the subsampled data maintaining the original distribution
    sub_top_1 = [sub_top_1 for _,sub_top_1 in sub_samples]
   
    print(len(sub_top_1))
    """ 
    plt.hist(sub_top_1,edgecolor='black')
    plt.title(f'Top-1 softmax Histogram for class {i} and subsamples')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.savefig(f'sub_top_1_hist_class_{i}.png')
    plt.show()
    """
    label_1 = []
    for j in range(29):
        label_1.append(i)
    y=  tf.keras.utils.to_categorical(label_1, num_classes=10)
    ds_true= tf.data.Dataset.from_tensor_slices(([x.numpy() for x,_ in sub_samples], y))
    new_ds_true = ds_true.map(reshape_y)
    
    # find 1 image from samples_false
    pred_labels_false = [np.argmax(sample[1]) for sample in samples_false]
    counter = Counter(pred_labels_false)
    common_label, common_count = counter.most_common(1)[0]
    print(f"for class{i} most common misclassified class= {common_label},count={common_count}")
    list_false=[]
    for sample in samples_false:
        if np.argmax(sample[1]) == common_label:
            list_false.append(sample)
            break 
    
    label_2 = tf.keras.utils.to_categorical(i, num_classes=10)
    a = []
    a.append(label_2)
    image = [sample[0] for sample in list_false]
    ds_false = tf.data.Dataset.from_tensor_slices((image,a)) 
    new_ds_false =  ds_false.map(reshape_y) 
    
    sampled_test = new_ds_true.concatenate(new_ds_false)
    arr = []
    
    for x,_ in sampled_test:
        arr.append(x.numpy())
    output_zip_path = f'/kaggle/working/ds_test_subsampled_{i}'
    save_images(arr, output_zip_path, image_format='PNG')
   

    sampled_datasets.append(sampled_test)
        
ds_test_sampled = sampled_datasets[0]
for ds in sampled_datasets[1:]:
    ds_test_sampled = ds_test_sampled.concatenate(ds) 
    

    
    


In [ ]:
import tensorflow as tf
import numpy as np
import random

classes = [0,1,2,3,4,5,6,7,8]

def get_unique_images(classes,dataset):
    labels = tf.keras.utils.to_categorical(classes, num_classes=10)
    images_labels = []
    for x, y in dataset:
        if y in labels and not any(np.array_equal(y.numpy(), item[1].numpy()) for item in images_labels): 
            images_labels.append((x, y))
        if len([item[1] for item in images_labels]) == len(classes):
            break
    
    return images_labels
    

In [ ]:
classes = [0,1,2,3,4,5,6,7,8]

def take_samples(num, dataset):
    labels = tf.keras.utils.to_categorical(classes, num_classes=10)
    images_labels = []
    samples_per_class = {cls: 0 for cls in classes}
    for x, y in dataset:
        if y in labels and samples_per_class[np.argmax(y)] < num:
            images_labels.append((x, y))
            samples_per_class[np.argmax(y)] += 1
        if all(count == num for count in samples_per_class.values()):
              break
    
    return images_labels

In [ ]:
class PrintLossCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: loss = {logs['loss']}, accuracy = {logs['accuracy']}")
    
    def on_batch_end(self, batch, logs=None):
        print(f"Batch {batch+1}: loss = {logs['loss']}")

In [ ]:
import time
Time = []
def AllTests(): 
    model = load_model("/kaggle/input/cifar9_95acc/tensorflow2/cifar9_95acc/1/mnv2_cifar9_160_fbn_4.keras")
    opt = keras.optimizers.Adam(learning_rate=0.000005, epsilon=0.002, amsgrad=True, weight_decay=1e-5)
    loss = keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
    model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
    loss_n=[]
    acc_n = []
    acc_nm = []
    acc_n_stratified = []
    acc_n_subsampling = []
    acc_m = []
    loss_nm = []
    loss_n_stratified = []
    loss_n_subsampling = []
    loss_m = []
    counter = 1e10
    i = 0
    c = 0 # for counting the total number of training steps
    case = True
    a = 0 
    dataset_n = take_samples(5, ds_train_n)
    m_dict = {}
    var = False 
    while(case):
        
        print(i+1)
        m_data = ds_train_m.take(1)
        
        if m_data not in list(m_dict.keys()):
            
            ds_n = random.sample(dataset_n, 4)
            x_n = [item[0] for item in ds_n]
            y_n = [item[1] for item in ds_n]
            ds_n = tf.data.Dataset.from_tensor_slices((x_n,y_n))
            ds = m_data.concatenate(ds_n)
            
            start_time = time.time()
            history = model.fit(ds, epochs=1, verbose=0)
            end_time = time.time()
            elapsed_time = end_time - start_time
            Time.append(elapsed_time)
            c += 1
            
            k = random.randint(5, 10)
            m_dict[m_data] = i+k # next i that this image needs to be used again

            loss, accuracy_n = model.evaluate(ds_test_n, verbose=0)
            print("Test accuracy on n: {}%".format(round(accuracy_n*100, 2)))
            acc_n.append(accuracy_n)
            loss_n.append(loss)
            
            # Stratified Sampling 
            loss_n_str, accuracy_n_stratified = model.evaluate(ds_test_stratified, verbose=0)
            print("Test accuracy on n for stratified Sampling: {}%".format(round(accuracy_n_stratified*100, 2)))
            acc_n_stratified.append(accuracy_n_stratified)
            loss_n_stratified.append(loss_n_str)
            
             # Subsampling 
            loss_n_Sub , accuracy_n_sub = model.evaluate(ds_test_sampled, verbose=0)
            print("Test accuracy on n for subsampling: {}%".format(round(accuracy_n_sub*100, 2)))
            acc_n_subsampling.append(accuracy_n_sub)
            loss_n_subsampling.append(loss_n_Sub)

            loss_m_, accuracy_m = model.evaluate(ds_test_m, verbose=0)
            print("Test accuracy on m: {}%".format(round(accuracy_m*100, 2)))
            acc_m.append(accuracy_m)
            loss_m.append(loss_m_)
            
            # check ending condition
            if abs(accuracy_n - accuracy_m) <= 0.02:
                print("For Original Test set, the accuracies were equal in iteration {} with total training steps={}, accuracy={}, and m_samples={}".format(i+1, c,accuracy_n, len(m_dict.keys())))
                if not var:
                    counter =  i + 5
                var = True  
            if abs(accuracy_n_stratified - accuracy_m) <= 0.02:
                print("For Stratified sampling, the accuracies were equal in iteration {} with total training steps={}, accuracy={}, and m_samples={}".format(i+1, c, accuracy_m, len(m_dict.keys())))
                if not var:
                    counter =  i + 5
                var = True 
            if abs(accuracy_n_sub - accuracy_m) <= 0.02:
                print("For Subsampling, the accuracies were equal in iteration {} with total training steps={}, accuracy={}, and m_samples={}".format(i+1, c, accuracy_m, len(m_dict.keys())))
                if not var:
                    counter =  i + 5
                var = True 
            # check stored images from m
            for m_data, next_i in m_dict.items():       
                if next_i == i:
                    
                    # update i
                    k = random.randint(5, 10)
                    m_dict[m_data] += k # next i that this image needs to be used again
                    
                    # generate a new 4-sample dataset from n-dataset again
                    ds_n = random.sample(dataset_n, 4)
                    x_n = [item[0] for item in ds_n]
                    y_n = [item[1] for item in ds_n]
                    ds_n = tf.data.Dataset.from_tensor_slices((x_n,y_n))
                    ds_new = m_data.concatenate(ds_n)
                    
                    # fit
                    start_time = time.time()
                    history = model.fit(ds_new, epochs=1, verbose=0)
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    Time.append(elapsed_time)
                    c += 1
                    
                    loss, accuracy_n = model.evaluate(ds_test_n, verbose=0)
                    print("Test accuracy on n: {}%".format(round(accuracy_n*100, 2)))
                    acc_n.append(accuracy_n)
                    loss_n.append(loss)
                    
                     # Stratified Sampling 
                    loss_n_str, accuracy_n_stratified = model.evaluate(ds_test_stratified, verbose=0)
                    print("Test accuracy on n for stratified Sampling: {}%".format(round(accuracy_n_stratified*100, 2)))
                    acc_n_stratified.append(accuracy_n_stratified)
                    loss_n_stratified.append(loss_n_str)
                    
                    # Subsampling 
                    loss_n_Sub , accuracy_n_sub = model.evaluate(ds_test_sampled, verbose=0)
                    print("Test accuracy on n for subsampling: {}%".format(round(accuracy_n_sub*100, 2)))
                    acc_n_subsampling.append(accuracy_n_sub)
                    loss_n_subsampling.append(loss_n_Sub)

                    loss_m_, accuracy_m = model.evaluate(ds_test_m, verbose=0)
                    print("Test accuracy on m: {}%".format(round(accuracy_m*100, 2)))
                    acc_m.append(accuracy_m)
                    loss_m.append(loss_m_)

                    if abs(accuracy_n - accuracy_m) <= 0.02:
                        print("For Original Test set, the accuracies were equal in iteration {} with total training steps={}, accuracy={}, and m_samples={}".format(i+1, c,accuracy_n, len(m_dict.keys())))
                        if not var:
                            counter = i + 5
                        var = True 
                    if abs(accuracy_n_stratified - accuracy_m) <= 0.02:
                        print("For Stratified sampling, the accuracies were equal in iteration {} with total training steps={}, accuracy={}, and m_samples={}".format(i+1, c, accuracy_m, len(m_dict.keys())))
                        if not var:
                            counter = i + 5
                        var = True 
                    if abs(accuracy_n_sub - accuracy_m) <= 0.02:
                        print("For Subsampling, the accuracies were equal in iteration {} with total training steps={}, accuracy={}, and m_samples={}".format(i+1, c, accuracy_m, len(m_dict.keys())))
                        if not var:
                            counter =  i + 5
                        var = True 
                    
        if i >= counter:  
            break
        i+=1
        print()
    return acc_n, acc_n_stratified, acc_n_subsampling, acc_m,loss_n, loss_n_stratified, loss_n_subsampling,loss_m,Time

In [ ]:
acc_n, acc_n_stratified, acc_n_subsampling, acc_m,loss_n, loss_n_stratified, loss_n_subsampling,loss_m,Time = AllTests()
plt.plot(acc_n, label="Original 9 classes")
plt.plot(acc_n_stratified, label="Stratified Sampling in 9 classes")
plt.plot(acc_n_subsampling, label="Stratified Sampling in 9 classes")
plt.plot(acc_m, label="1 class on Server")
plt.xlabel("Training Steps")  
plt.ylabel("Accuracy")
plt.legend()
plt.savefig('comparison_acc_all.png')
plt.show()


In [ ]:
del Time[0]
plt.plot(Time ,label='Training Time on Server')
#plt.plot(device_time ,label='Training Time on Device')
plt.xlabel("Traininng Steps")  
plt.ylabel("Time")
plt.legend()
plt.savefig('Time_all.png')
plt.show()
